In [1]:

## for data
import pandas as pd
import numpy as np
import re
import os
import glob

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from pathlib import Path

import torch
# import gradio as gr
from tqdm import tqdm
from time import sleep

from typing import Dict, List, Tuple
import codecs
from text_unidecode import unidecode

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 6000

Reading Traning data

In [4]:
# filepath = '/content/drive/MyDrive/fourthbrain/Effective_Writing_Feedback/data/train.csv'
# train_data = pd.read_csv(filepath)
# train_data.shape

In [5]:
import gc
gc.collect()

0

In [6]:
def read_data(path:str, file_type:str)-> pd.DataFrame:
    if glob.glob(path+file_type+".csv"):
        df = pd.read_csv(path+file_type+".csv")
    
    essay_dict = {}
    for file in glob.glob(path+file_type+'/*.txt'): 
        with open(file, "r") as file_open:
            filename = file.split(file_type+'/')[1].split('.txt')[0]
            essay_dict[filename] = file_open.read()
        
    essay_data = pd.DataFrame.from_dict([essay_dict]).T.reset_index()
    essay_data.columns = ["essay_id", "essay_text"]
    train_data = pd.merge(df,essay_data,left_on = 'essay_id', right_on ='essay_id', how ='left')
    print(file_type+" data: ",train_data.shape)
    return train_data

### Reading Train data

In [7]:
filepath = '/content/drive/MyDrive/fourthbrain/Effective_Writing_Feedback/data/'
train_data = read_data(filepath, 'train')
test_data = read_data(filepath, 'test')


train data:  (36765, 6)
test data:  (21, 5)


## Cleaning textual data

In [8]:
# https://www.kaggle.com/competitions/feedback-prize-2021/discussion/313330
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [10]:
def cleanup_text(text):
    punctuation_signs = list("?:!,;")
    text = text.replace("\n","")
    text = text.replace("\r", " ")
    text = text.replace("    ", " ")
    text = text.replace('"', '')
    text = text.replace("'s", "")
    for punct_sign in punctuation_signs:
      text = text.replace(punct_sign, '')
    words = re.sub(pattern = '[^a-zA-Z]',repl = ' ', string = text)
    words = words.lower()
    return words


train_data['processed_discourse'] = train_data['discourse_text'].apply(cleanup_text)
# train_data['processed_essay'] = train_data['essay_text'].apply(cleanup_text)


test_data['normalized_discourse'] = test_data['discourse_text'].apply(resolve_encodings_and_normalize)
test_data['processed_discourse'] = test_data['normalized_discourse'].apply(cleanup_text)

test_data['processed_essay'] = test_data['essay_text'].apply(resolve_encodings_and_normalize)
test_data['processed_essay'] = test_data['processed_essay'].apply(cleanup_text)
display(test_data[test_data['discourse_id'] == '84942fc2ab0f'].head(1))

,discourse_id,essay_id,discourse_text,discourse_type,essay_text,normalized_discourse,processed_discourse,processed_essay
12,84942fc2ab0f,A8445CABFECE,"According to an article by the Edgar Snyder Firm, 21% of teens that were part of a fatal car accident was due to phones. According to the same article, 35% know the risk but continue using their phones while on the road. This shows that its beyond dangerous and irresponsible of drivers not to be fully aware of their surroundings while driving. Drivers should be able to concentrate without any distractions, because it could be fatal.According to another article, ""Distracted Driving"" by the NHTSA, there has already been about 3,000 phone related car accident deaths since 2017. The article states that teen get too distracted with their phones, which causes their accident. Accidents that can be easily avoided by focusing on the road and not a phone. Drivers should not be able to use their phones at all while driving.",Evidence,"Phones & Driving\n\nDrivers should not be able to use phones while operating a vehicle. Drivers who used their phone while operating a vehicle are most likely to get into an accident that could be fatal.\n\nAccording to an article by the Edgar Snyder Firm, 21% of teens that were part of a fatal car accident was due to phones. According to the same article, 35% know the risk but continue using their phones while on the road. This shows that its beyond dangerous and irresponsible of drivers not to be fully aware of their surroundings while driving. Drivers should be able to concentrate without any distractions, because it could be fatal.\n\nAccording to another article, ""Distracted Driving"" by the NHTSA, there has already been about 3,000 phone related car accident deaths since 2017. The article states that teen get too distracted with their phones, which causes their accident. Accidents that can be easily avoided by focusing on the road and not a phone. Drivers should not be able to use their phones at all while driving.\n\nIn conclusion, drivers should not able to work a vehicle while using their cell phone. Drivers who uses their phones while operating a vehicle and are likely to have an accident then those who don't.","According to an article by the Edgar Snyder Firm, 21% of teens that were part of a fatal car accident was due to phones. According to the same article, 35% know the risk but continue using their phones while on the road. This shows that its beyond dangerous and irresponsible of drivers not to be fully aware of their surroundings while driving. Drivers should be able to concentrate without any distractions, because it could be fatal.According to another article, ""Distracted Driving"" by the NHTSA, there has already been about 3,000 phone related car accident deaths since 2017. The article states that teen get too distracted with their phones, which causes their accident. Accidents that can be easily avoided by focusing on the road and not a phone. Drivers should not be able to use their phones at all while driving.",according to an article by the edgar snyder firm of teens that were part of a fatal car accident was due to phones according to the same article know the risk but continue using their phones while on the road this shows that its beyond dangerous and irresponsible of drivers not to be fully aware of their surroundings while driving drivers should be able to concentrate without any distractions because it could be fatal according to another article distracted driving by the nhtsa there has already been about phone related car accident deaths since the article states that teen get too distracted with their phones which causes their accident accidents that can be easily avoided by focusing on the road and not a phone drivers should not be able to use their phones at all while driving,phones drivingdrivers should not be able to use phones while operating a vehicle drivers who used their phone while operating a vehicle are most like

In [11]:
def text_to_char_idx(full_text, substring):
  try:
    words = substring.split()
    essay_token = full_text.split()
    substring_text = " ".join(words)
    essay_text = " ".join(essay_token)
    start_char = essay_text.find(substring_text)
    end_char = start_char + len(substring_text)
    essay_len = len(essay_text)

  except Exception as e:
    print(e, full_text, substring)
  return start_char, end_char, essay_len

# def essay_len(full_text):
#   essay_token = full_text.split()
#   essay_text = " ".join(essay_token)
#   return len(essay_text)


In [12]:
test_data[['discourse_start','discourse_end','essay_len']]= test_data.apply(lambda x: text_to_char_idx( x['processed_essay'] ,x['processed_discourse']),\
                                                                axis=1 , result_type='expand')
# test_data['essay_len'] =test_data.apply(lambda x: essay_len(x['processed_essay']), axis = 1)

# test_data.drop(columns =['essay_text','processed_essay'], inplace = True)

In [13]:
test_data[test_data['discourse_start'] == -1]

,discourse_id,essay_id,discourse_text,discourse_type,essay_text,normalized_discourse,processed_discourse,processed_essay,discourse_start,discourse_end,essay_len


Calculating Gap length between 2 discourses

In [14]:
#initialize column
test_data['gap_length'] = np.nan
#set the first one
test_data.loc[0, 'gap_length'] = 0

#loop over rest
for i in range(1, len(test_data)):
    #gap if difference is not 1 within an essay
    if ((test_data.loc[i, "essay_id"] == test_data.loc[i-1, "essay_id"])\
        and (test_data.loc[i, "discourse_start"] - test_data.loc[i-1, "discourse_end"] > 1)):
        test_data.loc[i, 'gap_length'] = test_data.loc[i, "discourse_start"] - test_data.loc[i-1, "discourse_end"] - 2
        #minus 2 as the previous end is always -1 and the previous start always +1
    #gap if the first discourse of an new essay does not start at 0
    elif ((test_data.loc[i, "essay_id"] != test_data.loc[i-1, "essay_id"])\
        and (test_data.loc[i, "discourse_start"] != 0)):
        test_data.loc[i, 'gap_length'] = test_data.loc[i, "discourse_start"] -1


 #is there any text after the last discourse of an essay?
last_ones = test_data.drop_duplicates(subset="essay_id", keep='last')
last_ones['gap_end_length'] = np.where((last_ones.discourse_end < last_ones.essay_len),\
                                       (last_ones.essay_len - last_ones.discourse_end),\
                                       np.nan)

cols_to_merge = ['essay_id', 'gap_end_length']
test_data_w_gap_len = test_data.merge(last_ones[cols_to_merge], on = ["essay_id"], how = "left")

<ipython-input-14-123e11db70a2>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_ones['gap_end_length'] = np.where((last_ones.discourse_end < last_ones.essay_len),\


For testing purpose, I added label as **Effective** for all inputs

In [15]:
test_data_w_gap_len['label'] = 'Effective'

In [16]:
test_data_w_gap_len.head()

,discourse_id,essay_id,discourse_text,discourse_type,essay_text,normalized_discourse,processed_discourse,processed_essay,discourse_start,discourse_end,essay_len,gap_length,gap_end_length,label
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,Lead,"Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. Seeking multiple opinions can help a person make a better choice because it can decrease stress levels, a great chance to learn something new, can be very helpful and beneficial.\n\nTo begin with, Seeking information from more than one person can decrease stress levels. When making a decision there is a chance that you can become very stressed and overwhelmed. Being stressed can cause a person to give up on what they are doing. Having multiple opinions from people can make choosing an option less difficult and more easier. The information that you receive from others may have a special meaning in the future. Other peoples opinion can make a person feel confident in making the right choice.\n\nWhen you ask other people for their opinion you can also learn many new things. Everyone is different and may have more experience than you. Seeking other peoples opinion can cause you to gain a new skill or lesson. For example, someones advice could teach you how to do something the correct way. Many people are very different and have experience different things in life. Seeking advice from others can teach you a lot. We all learn from our mistakes in life, by sharing your past experiences you may prevent someone else from making the same mistake.\n\nSeeking others opinion can be very helpful and beneficial. Taking other peoples advice and doing what they say may lead to a great outcome. When you receive other opinions whether they are good or bad you will be able to look at them from a different point of view. For example, When a group of people give you advice on yourself, you then know how they look at you as a person. Everyone looks at certain things from a different prospective. How someone else looks at it may be how its supposed to be viewed in life.\n\nIn conclusion, You can learn from others experiences by seeking the advice that someone give you. Making decisions can cause stress on a person. Relating to someones advice may higher your self-esteem because you no longer feel left out. Ask others for advice when making a difficult decision. When you seek others opinion more opportunities are available fro yourself.",Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,making choices in life can be very difficult people often ask for advice when they can not decide on one thing it always good to ask others for their advice when making a choice when you have multiple opinions you have the ability to make the best choice for yourself,making choices in life can be very difficult people often ask for advice when they can not decide on one thing it always good to ask others for their advice when making a choice when you have multiple opinions you have the ability to make the best choice for yourself seeking multiple opinions can help a person make a better choice because it can decrease stress levels a great chance to learn something new can be very helpful and beneficial to begin with seeking information from more than one person can decrease stress levels w

In [17]:
def add_gap_rows(essay):
    cols_to_keep = ['discourse_start', 'discourse_end', 'discourse_type', 'gap_length', 'gap_end_length', 'label']
    df_essay = test_data_w_gap_len.query('essay_id == @essay')[cols_to_keep].reset_index(drop = True)

    #index new row
    insert_row = len(df_essay)
   
    for i in range(1, len(df_essay)):          
        if df_essay.loc[i,"gap_length"] >0:
            if i == 0:
                start = 0 #as there is no i-1 for first row
                end = df_essay.loc[0, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                label = ""
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end, label]
                insert_row += 1
            else:
                start = df_essay.loc[i-1, "discourse_end"] + 1
                end = df_essay.loc[i, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                label = ""
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end, label]
                insert_row += 1

    df_essay = df_essay.sort_values(by = "discourse_start").reset_index(drop=True)

    #add gap at end
    if df_essay.loc[(len(df_essay)-1),'gap_end_length'] > 0:
        start = df_essay.loc[(len(df_essay)-1), "discourse_end"] + 1
        end = start + df_essay.loc[(len(df_essay)-1), 'gap_end_length']
        disc_type = "Nothing"
        gap_end = np.nan
        gap = np.nan
        label = ""
        df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end, label]

    df_essay.loc[df_essay.discourse_type.isna(),'discourse_type'] = 'Nothing'
    # print(df_essay[df_essay.discourse_type.isna()].shape)
    return(df_essay)

In [18]:
add_gap_rows("A8445CABFECE")

,discourse_start,discourse_end,discourse_type,gap_length,gap_end_length,label
0,14,80,Position,13.0,NaN,Effective
1,81,195,Claim,NaN,NaN,Effective
2,196,986,Evidence,NaN,NaN,Effective
3,987,1183,Concluding Statement,NaN,NaN,Effective


In [19]:
import spacy



def print_colored_essay(essay_file):
    essay_id = os.path.basename(essay_file).split('.')[0]
    df_essay = add_gap_rows(essay_id)
    ents = []
    for i, row in df_essay.iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                         'end': int(row['discourse_end']), 
                         'label': row['discourse_type']
                    })


    with open(essay_file, 'r') as file: 
      data = file.read()
      clean_data = cleanup_text(data)
      tokens_essay = clean_data.split()
      essay_text = " ".join(tokens_essay)

    doc = {"text": essay_text, "ents": ents,}
    print(doc)
    colors = {'Lead': '#EE11D0','Position': '#AB4DE1','Claim': '#1EDE71','Evidence': '#33FAFA','Counterclaim': '#4253C1',\
                 'Concluding Statement': 'yellow','Rebuttal': 'red'}
    options = {"ents": df_essay.discourse_type.unique().tolist(), "colors": colors}

    spacy.displacy.render(doc, style="ent", options=options, manual=True, jupyter=True);
 

In [20]:
essay_file = '/content/drive/MyDrive/fourthbrain/Effective_Writing_Feedback/data/test/A8445CABFECE.txt'
print_colored_essay(essay_file)

{'text': 'phones drivingdrivers should not be able to use phones while operating a vehicle drivers who used their phone while operating a vehicle are most likely to get into an accident that could be fatal according to an article by the edgar snyder firm of teens that were part of a fatal car accident was due to phones according to the same article know the risk but continue using their phones while on the road this shows that its beyond dangerous and irresponsible of drivers not to be fully aware of their surroundings while driving drivers should be able to concentrate without any distractions because it could be fatal according to another article distracted driving by the nhtsa there has already been about phone related car accident deaths since the article states that teen get too distracted with their phones which causes their accident accidents that can be easily avoided by focusing on the road and not a phone drivers should not be able to use their phones at all while driving in 

In [21]:
# def read_file(file_name,text):
#   try:
#     essay_id = os.path.basename(file_name).split('.')[0]
#     df_essay = add_gap_rows(essay_id)
    
#     ents = []
#     for i, row in df_essay.iterrows():
#         ents.append({
#                         'entity': row['discourse_type'] + ":: " +'Effective',
#                         'start': int(row['discourse_start']), 
#                          'end': int(row['discourse_end']), 
#                     })

#     clean_data = cleanup_text(text)
#     tokens_essay = clean_data.split()
#     essay_text = " ".join(tokens_essay)

#     doc = {"text": essay_text, "entities": ents}
#     return doc
#   except Exception as e:
#     return e


# demo = gr.Interface(read_file,
#              inputs = [gr.Textbox(placeholder="Enter sentence here...", lines = 1, label ="Enter Essay Id"),gr.Textbox(placeholder="Enter sentence here...", lines = 30, label ="Enter Essay Text")],
#              outputs = gr.HighlightedText(), examples=[["D72CB1C11673", essay_text_ex]])


# if __name__ == "__main__":
#     demo.launch()


In [22]:
# essay_file = '/content/drive/MyDrive/fourthbrain/Effective_Writing_Feedback/data/test/D72CB1C11673.txt'
# read_file(essay_file)

In [23]:
# curl \
# --header "Content-Type: application/json" \
# --request POST \
# --data '{"essay_text":"Seeking others opinion can be very helpful and beneficial."}' \
# http://50.18.96.207:8000/predict